# CrashWeather
----


In [25]:
%matplotlib inline

In [26]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime
from datetime import datetime
from datetime import date
import os
import scipy.stats as sts
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

## Generate Cities List

In [27]:
# generate city list from plane_crashes.csv

cities = ["Chicago", "New York"]
date = []

In [28]:
plane_crash_data = "data/plane_crashes.csv"
crash_data = pd.read_csv(plane_crash_data)
crash_data.head()

,Crash Id,Date,Location,City,Aboard,Fatalities,Summary,Year
0,0,01/02/1980,Haiti,Haiti,9.0,9.0,Struck high ground in poor visibility while en...,1980
1,1,01/11/1980,Atlantic Ocean,Atlantic Ocean,2.0,2.0,The plane was on a 45 minute fight from Shreve...,1980
2,2,01/14/1980,"Shungnak, Alaska",Shungnak,9.0,7.0,Crashed. Continued VFR flight into adverse wea...,1980
3,3,01/21/1980,"Elburz Mtns., Tehran, Iran",Elburz Mtns.,128.0,128.0,The aircraft crashed into a mountain during a ...,1980
4,4,01/23/1980,"Mt. Cemonyet, Indonesia",Mt. Cemonyet,13.0,13.0,"Hit high ground at 2,700 ft. in poor weather w...",1980


In [29]:
#cities = crash_data['City'].tolist()
dates = crash_data['Date'].tolist()
print(dates)

['01/02/1980', '01/11/1980', '01/14/1980', '01/21/1980', '01/23/1980', '01/24/1980', '02/03/1980', '02/21/1980', '02/21/1980', '02/22/1980', '02/27/1980', '03/14/1980', '03/14/1980', '03/21/1980', '03/21/1980', '04/12/1980', '04/14/1980', '04/24/1980', '04/25/1980', '04/27/1980', '06/02/1980', '06/08/1980', '06/12/1980', '06/12/1980', '06/27/1980', '07/08/1980', '08/01/1980', '08/06/1980', '08/07/1980', '08/19/1980', '08/26/1980', '09/05/1980', '09/10/1980', '09/12/1980', '09/14/1980', '09/15/1980', '09/17/1980', '09/30/1980', '11/12/1980', '11/19/1980', '11/24/1980', '11/28/1980', '12/05/1980', '12/21/1980', '12/23/1980', '01/14/1981', '01/20/1981', '02/07/1981', '02/11/1981', '02/22/1981', '02/24/1981', '02/26/1981', '03/10/1981', '03/15/1981', '03/26/1981', '04/06/1981', '04/07/1981', '04/17/1981', '04/28/1981', '04/28/1981', '05/06/1981', '05/07/1981', '05/16/1981', '05/21/1981', '05/22/1981', '05/29/1981', '06/14/1981', '06/26/1981', '07/18/1981', '07/20/1981', '07/24/1981', '07/2

In [30]:
u_dates = []
for date in dates:
    x=datetime.strptime(date, "%m/%d/%Y")
    x=time.mktime(x.timetuple())
    u_dates.append(int(x))
    

In [31]:
u_dates

[315640800,
 316418400,
 316677600,
 317282400,
 317455200,
 317541600,
 318405600,
 319960800,
 319960800,
 320047200,
 320479200,
 321861600,
 321861600,
 322466400,
 322466400,
 324367200,
 324540000,
 325404000,
 325490400,
 325663200,
 328770000,
 329288400,
 329634000,
 329634000,
 330930000,
 331880400,
 333954000,
 334386000,
 334472400,
 335509200,
 336114000,
 336978000,
 337410000,
 337582800,
 337755600,
 337842000,
 338014800,
 339138000,
 342856800,
 343461600,
 343893600,
 344239200,
 344844000,
 346226400,
 346399200,
 348300000,
 348818400,
 350373600,
 350719200,
 351669600,
 351842400,
 352015200,
 353052000,
 353484000,
 354434400,
 355384800,
 355471200,
 356335200,
 357282000,
 357282000,
 357973200,
 358059600,
 358837200,
 359269200,
 359355600,
 359960400,
 361342800,
 362379600,
 364280400,
 364453200,
 364798800,
 364885200,
 365058000,
 367304400,
 367477200,
 367736400,
 367909200,
 368254800,
 369637200,
 369896400,
 369982800,
 370587600,
 370760400,
 371

### Perform API Calls

* Perform a weather check on each city using a series of successive API calls.

* Include a print log of each city as it'sbeing processed (with the city number and city name).

In [32]:
# Save config information
url = "http://history.openweathermap.org/data/2.5/history/"
units = "imperial"

# Build partial query URL
query_url = f"{url}appid={weather_api_key}&units={units}&q="
#query_url = f"{city name},{country code}&type=day&start={start}&end={end}&appid={weather_api_key}"

In [40]:
get_url = query_url + city + "&" + "start=" + str(u_date)
response = requests.get(get_url).json()
print(get_url)

http://history.openweathermap.org/data/2.5/history/appid=66cd7594b845c1f2e8591cf7ad563279&units=imperial&q=New York&start=316418400


In [33]:
#counter for city numbers
counter = 1

#variables for weather data frame
date = []
city = []
temp = []
pressure = []
humidity = []
wind_speed = []
weather_summary = []
fog = []
cloudiness = []


print('Beginning City Data Retrieval')
print('------------------------------')
# Loop through the list of cities and perform a request for data on each
for city, u_date in zip(cities, u_dates):
    
    try:
        response = requests.get(query_url + city).json()
        date.append(response['dt'])
        city.append(response['name'])
        temp.append(response['main']['temp'])
        pressure.append(response['main']['pressure'])
        hummidity.append(response['main']['humidity'])
        wind_speed.append(response['wind']['speed'])
        weather_summary.append(response['main']['humidity'])
        fog.append(response['wind']['speed'])
        cloudiness.append(response['clouds']['all'])
        
        #print result city
        print(f'Processing Record #{counter} of #{len(cities)} initial results | {city} on date of {u_date}')
        #set counter
        counter = counter + 1
    except (KeyError, IndexError):
        print(f"City data for record #{counter} not found. Skipping...")
        counter = counter


#("city")str.title()
print('------------------------------')
print(f"End of City Data Retrieval. Total Weather Records Retrieved from original {len(cities)} is {counter}")
print('------------------------------')

Beginning City Data Retrieval
------------------------------
City data for record #1 not found. Skipping...
City data for record #1 not found. Skipping...
------------------------------
End of City Data Retrieval. Total Weather Records Retrieved from original 2 is 1
------------------------------


### Convert Raw Data to DataFrame

In [ ]:
city_data_df = pd.DataFrame({'City': city_name,
                             'Country': country,
                            'Latitude': lat,
                            'Longitude': long,
                            'Temp': temp_imperial,
                            'Humidity': humidity,
                            'Cloudiness': cloudiness,
                            'Wind Speed': wind_speed
                            })

* Export the city data into a .csv.

In [ ]:
#pd.DataFrame.to_csv(city_data_df, '../output_data/city_weather_data.csv')

* Display the DataFrame

In [ ]:
city_data_df.head()

In [ ]:
len(city_data_df)

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#Get the indices of cities that have humidity over 100%.
#checking for High Humidity Cities and confirming visually
city_data_h_humid = city_data_df[city_data_df['Humidity'] > 99]
city_data_h_humid

In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


In [ ]:
#Dropping rows of the High Humidity Cities if they exist and confirming the database count is lower.
clean_city_data = city_data_df.drop(city_data_df[city_data_df['Humidity'] > 99].index, inplace = False)
len(clean_city_data)

In [ ]:
# Extract relevant fields from the data frame
clean_city_data.reset_index(drop=True, inplace=True)
clean_city_data

In [ ]:
# Export the City_Data into a csv
#pd.DataFrame.to_csv(clean_city_data, '../output_data/clean_city_data.csv')

In [ ]:
pd.DataFrame.to_html(clean_city_data, '../output_data/clean_city_data.html')

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.

* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
#I perfer to drop the alpha a bit. The overlay of the plots display a nice density.
plt.scatter(clean_city_data['Latitude'], clean_city_data['Temp'],
            marker="o", s=50, facecolors='#E99202', edgecolors='#754901', alpha=0.6)
plt.title(f"City Latitude vs. Temperature ({date.today()})", fontsize=14)
plt.xlabel("Latitude", fontsize=12)
plt.ylabel("Temp (F)", fontsize=12)
plt.grid(True)
plt.savefig('../images/lat_temp.png', bbox_inches='tight')
plt.show()


# Analysis of the Latitude vs. Temp Plot
* This scatter plot clearly depicts the rise of temperature from the Southern Hemisphere's lowest latitude (-60) to the equator (0) and the Northern's highest latitude (80) to the equater (0).
* We can also cleary see there are more cities, in general, in the Northern Hemisphere.

## Latitude vs. Humidity Plot

In [ ]:
plt.scatter(clean_city_data['Latitude'], clean_city_data['Humidity'],
            marker="o", s=50, facecolors='#027886', edgecolors='#003036', alpha=0.6)
plt.title(f"City Latitude vs. Humidity ({date.today()})", fontsize=14)
plt.xlabel("Latitude", fontsize=12)
plt.ylabel("Humidity (%)", fontsize=12)
plt.grid(True)
plt.savefig('../images/lat_humidity.png', bbox_inches='tight')
plt.show()

# Analysis of the Latitude vs. Humidity Plot
* This scatter plot shows that humidity level, world wide, has low correlation to latitude.
* Lower levels Humiduty and higher variance of Humidity, are recorded closer to the equator.

## Latitude vs. Cloudiness Plot

In [ ]:
plt.scatter(clean_city_data['Latitude'], clean_city_data['Cloudiness'],
            marker="o", s=50, facecolors='#9DA5B9', edgecolors='#575C69', alpha=0.8)
plt.title(f"City Latitude vs. Cloudiness ({date.today()})", fontsize=14)
plt.xlabel("Latitude", fontsize=12)
plt.ylabel("Cloudiness", fontsize=12)
plt.grid(True)
plt.savefig('../images/lat_clouds.png', bbox_inches='tight')
plt.show()

# Analysis of the Latitude vs. Cloudiness Plot
* This scatter plot shows that Cloudiness level, world wide, has low correlation to latitude.

## Latitude vs. Wind Speed Plot

In [ ]:
plt.scatter(clean_city_data['Latitude'], clean_city_data['Wind Speed'],
            marker="o", s=50, facecolors='#33747C', edgecolors='#1B3C40', alpha=0.8)
plt.title(f"City Latitude vs. Wind Speed ({date.today()})", fontsize=14)
plt.xlabel("Latitude", fontsize=12)
plt.ylabel("Wind Speed (mph)", fontsize=12)
plt.grid(True)
plt.savefig('../images/lat_wind.png', bbox_inches='tight')
plt.show()

# Analysis of the Latitude vs. Wind Speed Plot
* This scatter plot shows that Wind Speed, world wide, has low correlation to latitude.

## Linear Regression Latitude vs. Temperature Plot

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots

In [ ]:
# Calculate the correlation coefficient and linear regression model 
lat_line = clean_city_data.iloc[:,2]
temp_lin = clean_city_data.iloc[:,4]
correlation = sts.pearsonr(lat_line,temp_lin)

#sets a variable for using '\n' in printed 'fstring'
nreturn = '\n'

x_values = clean_city_data['Latitude']
y_values = clean_city_data['Temp']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.title(f"City Latitude vs. Temperature ({date.today()})", fontsize=14)
plt.xlabel("Latitude", fontsize=12)
plt.ylabel("Temperature", fontsize=12)
plt.grid(True)
#plt.tight_layout()
plt.scatter(x_values,y_values, marker="o", s=50, facecolors='#E99202', edgecolors='#754901', alpha=0.6)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")
print(f"The correlation between Latitude {nreturn}and Temperature is {round(correlation[0],2)}")

plt.savefig("../images/lat_temp_lin_regress.png", bbox_inches = 'tight', dpi=100)
plt.show()


# Analysis: Linear regression of the City Latitude vs Temperature plot
* The data in this plot depicts no "standard" linear regression. 
* In this case the plot depicts two distinct arc values in the plotted data. 
* We would be best showing multi-regression lines on this plot for each hemisphere's latitude range.

# Create Northern and Southern Hemisphere DataFrames

In [ ]:
#Southern Latitude Database creation
slat_city_data = clean_city_data.drop(clean_city_data[clean_city_data['Latitude'] >= 0].index, inplace = False)
len(slat_city_data)

In [ ]:
# Extract relevant fields from the southern lat dataframe
slat_city_data.reset_index(drop=True, inplace=True)
slat_city_data

In [ ]:
#Northern Latitude Database creation
nlat_city_data = clean_city_data.drop(clean_city_data[clean_city_data['Latitude'] <= 0].index, inplace = False)
len(nlat_city_data)

In [ ]:
# Extract relevant fields from the northern lat dataframe
nlat_city_data.reset_index(drop=True, inplace=True)
nlat_city_data

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
from scipy.stats import linregress
# Calculate the correlation coefficient and linear regression model 
lat_line = nlat_city_data.iloc[:,2]
temp_lin = nlat_city_data.iloc[:,4]
correlation = sts.pearsonr(lat_line,temp_lin)

#sets a variable for using '\n' in printed 'fstring'
nreturn = '\n'

x_values = nlat_city_data['Latitude']
y_values = nlat_city_data['Temp']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.title(f"Northern Hemisphere City vs. Temperature ({date.today()})", fontsize=12)
plt.xlabel("Latitude", fontsize=12)
plt.ylabel("Temp (F)", fontsize=12)
plt.grid(True)
plt.scatter(x_values,y_values, marker="o", s=50, facecolors='#E99202', edgecolors="#754901", alpha=0.6)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")
#plt.tight_layout()
print(f"The correlation between N. Hemisphere {nreturn}and Temperature is {round(correlation[0],2)}")

plt.savefig("../images/nhem_temp_lin_regress.png", bbox_inches = 'tight', dpi=150)
plt.show()


# Analysis: N. Hemisphere Linear regression of Latitude vs Temperature plot
* The data shows a moderate linear relationship in temperature. The temp lowering as you get father away from the equator. 

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
from scipy.stats import linregress
# Calculate the correlation coefficient and linear regression model 
lat_line = slat_city_data.iloc[:,2]
temp_lin = slat_city_data.iloc[:,4]
correlation = sts.pearsonr(lat_line,temp_lin)

#sets a variable for using '\n' in printed 'fstring'
nreturn = '\n'

x_values = slat_city_data['Latitude']
y_values = slat_city_data['Temp']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.title(f"Southern Hemisphere City vs. Temperature ({date.today()})", fontsize=12)
plt.xlabel("Latitude", fontsize=12)
plt.ylabel("Temp (F)", fontsize=12)
plt.grid(True)
plt.scatter(x_values,y_values, marker="o", s=50, facecolors='#E99202', edgecolors="#754901", alpha=0.6)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")
#plt.tight_layout()
print(f"The correlation between S. Hemisphere {nreturn}and Temperature is {round(correlation[0],2)}")

plt.savefig("../images/shem_temp_lin_regress.png", bbox_inches = 'tight', dpi=150)
plt.show()


# Analysis: S. Hemisphere Linear regression of Latitude vs Temperature plot
* The data shows a strong linear relationship in temperature. The temp rising as you get closer to the equator.
* This Southern linear relationship (0.8) may be slightly stronger than the Northern's linear relationship (-0.68), because of the winter and less cities. More research should be done on this possible correlation.

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
lat_line = nlat_city_data.iloc[:,2]
humidity_lin = nlat_city_data.iloc[:,5]
correlation = sts.pearsonr(lat_line,humidity_lin)

#sets a variable for using '\n' in printed 'fstring'
nreturn = '\n'

x_values = nlat_city_data['Latitude']
y_values = nlat_city_data['Humidity']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.title(f"Northern Hemisphere City vs. Humidity ({date.today()})", fontsize=12)
plt.xlabel("Latitude", fontsize=12)
plt.ylabel("Humidity (%)", fontsize=12)
plt.grid(True)
plt.scatter(x_values,y_values, marker="o", s=50, facecolors='#027886', edgecolors="#003036", alpha=0.6)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")
#plt.tight_layout()
print(f"The correlation between N. Hemisphere {nreturn}and Humidity is {round(correlation[0],2)}")

plt.savefig("../images/nhem_humidity_lin_regress.png", bbox_inches = 'tight', dpi=150)
plt.show()


# Analysis: N. Hemisphere Linear regression of Latitude vs Humidity plot
* The data shows a weak linear relationship in Humidity. .
* This is not suprising because humidity has high vaiance world wide. 

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
lat_line = slat_city_data.iloc[:,2]
humidity_lin = slat_city_data.iloc[:,5]
correlation = sts.pearsonr(lat_line,humidity_lin)

#sets a variable for using '\n' in printed 'fstring'
nreturn = '\n'

x_values = slat_city_data['Latitude']
y_values = slat_city_data['Humidity']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.title(f"Southern Hemisphere City vs. Humidity ({date.today()})", fontsize=12)
plt.xlabel("Latitude", fontsize=12)
plt.ylabel("Humidity (%)", fontsize=12)
plt.grid(True)
plt.scatter(x_values,y_values, marker="o", s=50, facecolors='#027886', edgecolors="#003036", alpha=0.6)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")
#plt.tight_layout()
print(f"The correlation between S. Hemisphere {nreturn}and Humidity is {round(correlation[0],2)}")

plt.savefig("../images/shem_humidity_lin_regress.png", bbox_inches = 'tight', dpi=150)
plt.show()


# Analysis: S. Hemisphere Linear regression of Latitude vs Humidity plot
* The data shows a weak linear relationship in Humidity. .
* This is not suprising because humidity has high variance world wide. 

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
lat_line = nlat_city_data.iloc[:,2]
cloud_lin = nlat_city_data.iloc[:,6]
correlation = sts.pearsonr(lat_line,cloud_lin)

#sets a variable for using '\n' in printed 'fstring'
nreturn = '\n'

x_values = nlat_city_data['Latitude']
y_values = nlat_city_data['Cloudiness']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.title(f"Northern Hemisphere City vs. Cloudiness ({date.today()})", fontsize=12)
plt.xlabel("Latitude", fontsize=12)
plt.ylabel("Cloudiness", fontsize=12)
plt.grid(True)
plt.scatter(x_values,y_values, marker="o", s=50, facecolors='#9DA5B9', edgecolors="#575C69", alpha=0.6)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")
#plt.tight_layout()
print(f"The correlation between N. Hemisphere {nreturn}and Cloudiness is {round(correlation[0],2)}")

plt.savefig("../images/nhem_cloudiness_regress.png", bbox_inches = 'tight', dpi=150)
plt.show()


# Analysis: N. Hemisphere Linear regression of Latitude vs Cloudiness plot
* The data shows a weak linear relationship in Cloudiness. .
* This is not suprising because Cloudiness has high variance world wide. 

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
lat_line = slat_city_data.iloc[:,2]
cloud_lin = slat_city_data.iloc[:,6]
correlation = sts.pearsonr(lat_line,cloud_lin)

#sets a variable for using '\n' in printed 'fstring'
nreturn = '\n'

x_values = slat_city_data['Latitude']
y_values = slat_city_data['Cloudiness']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.title(f"Southern Hemisphere City vs. Cloudiness ({date.today()})", fontsize=12)
plt.xlabel("Latitude", fontsize=12)
plt.ylabel("Cloudiness", fontsize=12)
plt.grid(True)
plt.scatter(x_values,y_values, marker="o", s=50, facecolors='#9DA5B9', edgecolors="#575C69", alpha=0.6)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")
#plt.tight_layout()
print(f"The correlation between S. Hemisphere {nreturn}and Cloudiness is {round(correlation[0],2)}")

plt.savefig("../images/shem_cloudiness_regress.png", bbox_inches = 'tight', dpi=150)
plt.show()


# Analysis: S. Hemisphere Linear regression of Latitude vs Cloudiness plot
* The data shows a weak linear relationship in Cloudiness. .
* This is not suprising because Cloudiness has high variance world wide. 

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
lat_line = nlat_city_data.iloc[:,2]
cloud_lin = nlat_city_data.iloc[:,7]
correlation = sts.pearsonr(lat_line,cloud_lin)

#sets a variable for using '\n' in printed 'fstring'
nreturn = '\n'

x_values = nlat_city_data['Latitude']
y_values = nlat_city_data['Wind Speed']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.title(f"Northern Hemisphere City vs. Wind Speed ({date.today()})", fontsize=12)
plt.xlabel("Latitude", fontsize=12)
plt.ylabel("Wind Speed (mph)", fontsize=12)
plt.grid(True)
plt.scatter(x_values,y_values, marker="o", s=50, facecolors='#33747C', edgecolors='#1B3C40', alpha=0.6)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")
#plt.tight_layout()
print(f"The correlation between N. Hemisphere {nreturn}and Wind Speed is {round(correlation[0],2)}")

plt.savefig("../images/nhem_wind_regress.png", bbox_inches = 'tight', dpi=150)
plt.show()


# Analysis: N. Hemisphere Linear regression of Latitude vs Wind Speed plot
* The data shows a weak linear relationship in Wind Speed. .
* This is not suprising because Wind Speed has high variance world wide.
* There seems to be some higher rates of Wind Speed near the equator

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
lat_line = slat_city_data.iloc[:,2]
cloud_lin = slat_city_data.iloc[:,7]
correlation = sts.pearsonr(lat_line,cloud_lin)

#sets a variable for using '\n' in printed 'fstring'
nreturn = '\n'

x_values = slat_city_data['Latitude']
y_values = slat_city_data['Wind Speed']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.title(f"Southern Hemisphere City vs. Wind Speed ({date.today()})", fontsize=12)
plt.xlabel("Latitude", fontsize=12)
plt.ylabel("Wind Speed (mph)", fontsize=12)
plt.grid(True)
plt.scatter(x_values,y_values, marker="o", s=50, facecolors='#33747C', edgecolors='#1B3C40', alpha=0.6)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")
#plt.tight_layout()
print(f"The correlation between S. Hemisphere {nreturn}and Wind Speed is {round(correlation[0],2)}")

plt.savefig("../images/shem_wind_regress.png", bbox_inches = 'tight', dpi=150)
plt.show()


# Analysis: S. Hemisphere Linear regression of Latitude vs Wind Speed plot
* The data shows a weak linear relationship in Wind Speed. .
* This is not suprising because Wind Speed has high variance world wide.
* There seems to be some higher rates of Wind Speed near the South Pole. This could be because it is currently winter in the Southern Hemisphere.